In [43]:
import pymongo
from pymongo import MongoClient
import json
import tweepy
import twitter
from pprint import pprint
import configparser

In [44]:
config = configparser.ConfigParser()
config.read('config.ini')

CONSUMER_KEY      = config['mytwitter']['api_key']
CONSUMER_SECRET   = config['mytwitter']['api_secrete']
OAUTH_TOKEN       = config['mytwitter']['access_token']
OATH_TOKEN_SECRET = config['mytwitter']['access_secrete']

mongod_connect = config['mymongo']['connection']


In [45]:
client = MongoClient(mongod_connect)
db = client.tweet_db # create a database named tweet_db
tweet_collection = db.tweet_collection_job #create a collection called tweet_collection
tweet_collection.create_index([("id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'id_1'

In [46]:
!pip install dnspython

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [47]:

stream_auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
stream_auth.set_access_token(OAUTH_TOKEN, OATH_TOKEN_SECRET)

strem_api = tweepy.API(stream_auth)

In [48]:
track = ['ia'] # define the keywords, tweets contain JMU

In [ ]:
class MyStreamListener(tweepy.StreamListener):   
    def on_status(self, status):
        print (status.id_str)
        try:
            tweet_collection.insert_one(status._json)
        except:
            pass
  
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_data disconnects the stream
            return False
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = strem_api.auth, listener=myStreamListener)
myStream.filter(track=track)

1191810863465975814
1191810865311469569
1191810866917851137
1191810867651846146
1191810868151017473
1191810868322930688
1191810868595572736
1191810870529138688
1191810872982851584
1191810873033216006
1191810874350043136
1191810877219131395
1191810877323960320
1191810877600751616
1191810877680496641
1191810879047831558
1191810879425302533
1191810879706128385
1191810879978954760
1191810883703300096
1191810885322444801
1191810886895308801
1191810887763578880
1191810888027820032
1191810889298497536
1191810891551006727
1191810891681091585
1191810892289187845
1191810893866291202
1191810895510413312
1191810896810647554
1191810896991010818
1191810897968336897
1191810898295500801
1191810898849095682
1191810898882682880
1191810899335680001
1191810899666964481
1191810902795980800
1191810904209408011
1191810907178975233
1191810908135329794
1191810908147929094
1191810908592492545
1191810910404431878
1191810911494905860
1191810911474003975
1191810912539357184
1191810913533382658
1191810916444200962


In [32]:
print(tweet_collection.estimated_document_count())# number of tweets collected

user_cursor = tweet_collection.distinct("user.id")
print (len(user_cursor)) # number of unique Twitter users

0
0


In [33]:
rest_auth = twitter.oauth.OAuth(OAUTH_TOKEN,OATH_TOKEN_SECRET,CONSUMER_KEY,CONSUMER_SECRET)
rest_api = twitter.Twitter(auth=rest_auth)

In [34]:
count = 100 #number of returned tweets, default and max is 100
q = "ia job"                               #define the keywords, tweets contain JMU

In [35]:
search_results = rest_api.search.tweets( count=count,q=q,)
statuses = search_results["statuses"]
since_id_new = statuses[-1]['id']
for statuse in statuses:
    try:
        tweet_collection.insert_one(statuse)
        pprint(statuse['created_at'])# print the date of the collected tweets
    except:
        pass

'Tue Nov 05 20:01:45 +0000 2019'
'Tue Nov 05 20:01:36 +0000 2019'
'Tue Nov 05 19:56:34 +0000 2019'
'Tue Nov 05 19:55:54 +0000 2019'
'Tue Nov 05 19:55:13 +0000 2019'
'Tue Nov 05 19:54:54 +0000 2019'
'Tue Nov 05 19:54:43 +0000 2019'
'Tue Nov 05 19:50:34 +0000 2019'
'Tue Nov 05 19:50:24 +0000 2019'
'Tue Nov 05 19:49:57 +0000 2019'
'Tue Nov 05 19:48:43 +0000 2019'
'Tue Nov 05 19:46:52 +0000 2019'
'Tue Nov 05 19:46:43 +0000 2019'
'Tue Nov 05 19:46:18 +0000 2019'
'Tue Nov 05 19:45:33 +0000 2019'
'Tue Nov 05 19:45:13 +0000 2019'
'Tue Nov 05 19:44:52 +0000 2019'
'Tue Nov 05 19:44:23 +0000 2019'
'Tue Nov 05 19:42:21 +0000 2019'
'Tue Nov 05 19:39:12 +0000 2019'
'Tue Nov 05 19:38:32 +0000 2019'
'Tue Nov 05 19:37:22 +0000 2019'
'Tue Nov 05 19:30:41 +0000 2019'
'Tue Nov 05 19:29:21 +0000 2019'
'Tue Nov 05 19:28:53 +0000 2019'
'Tue Nov 05 19:28:22 +0000 2019'
'Tue Nov 05 19:27:20 +0000 2019'
'Tue Nov 05 19:25:30 +0000 2019'
'Tue Nov 05 19:24:02 +0000 2019'
'Tue Nov 05 19:23:37 +0000 2019'
'Tue Nov 0

In [36]:
since_id_old = 0
while(since_id_new != since_id_old):
    since_id_old = since_id_new
    search_results = rest_api.search.tweets( count=count,q=q,
                        max_id= since_id_new)
    statuses = search_results["statuses"]
    since_id_new = statuses[-1]['id']
    for statuse in statuses:
        try:
            tweet_collection.insert_one(statuse)
            pprint(statuse['created_at']) # print the date of the collected tweets
        except:
            pass

'Tue Nov 05 17:08:46 +0000 2019'
'Tue Nov 05 17:06:47 +0000 2019'
'Tue Nov 05 17:02:58 +0000 2019'
'Tue Nov 05 17:01:56 +0000 2019'
'Tue Nov 05 17:00:49 +0000 2019'
'Tue Nov 05 16:59:55 +0000 2019'
'Tue Nov 05 16:59:35 +0000 2019'
'Tue Nov 05 16:58:06 +0000 2019'
'Tue Nov 05 16:57:05 +0000 2019'
'Tue Nov 05 16:55:54 +0000 2019'
'Tue Nov 05 16:50:54 +0000 2019'
'Tue Nov 05 16:47:05 +0000 2019'
'Tue Nov 05 16:46:21 +0000 2019'
'Tue Nov 05 16:40:08 +0000 2019'
'Tue Nov 05 16:37:14 +0000 2019'
'Tue Nov 05 16:33:52 +0000 2019'
'Tue Nov 05 16:33:33 +0000 2019'
'Tue Nov 05 16:33:25 +0000 2019'
'Tue Nov 05 16:32:42 +0000 2019'
'Tue Nov 05 16:32:15 +0000 2019'
'Tue Nov 05 16:25:57 +0000 2019'
'Tue Nov 05 16:25:02 +0000 2019'
'Tue Nov 05 16:24:01 +0000 2019'
'Tue Nov 05 16:23:22 +0000 2019'
'Tue Nov 05 16:22:02 +0000 2019'
'Tue Nov 05 16:19:42 +0000 2019'
'Tue Nov 05 16:15:20 +0000 2019'
'Tue Nov 05 16:11:51 +0000 2019'
'Tue Nov 05 16:11:50 +0000 2019'
'Tue Nov 05 16:09:00 +0000 2019'
'Tue Nov 0

'Mon Nov 04 21:13:22 +0000 2019'
'Mon Nov 04 21:12:57 +0000 2019'
'Mon Nov 04 21:11:47 +0000 2019'
'Mon Nov 04 21:10:04 +0000 2019'
'Mon Nov 04 21:09:29 +0000 2019'
'Mon Nov 04 21:02:22 +0000 2019'
'Mon Nov 04 21:02:11 +0000 2019'
'Mon Nov 04 21:00:01 +0000 2019'
'Mon Nov 04 20:58:01 +0000 2019'
'Mon Nov 04 20:57:51 +0000 2019'
'Mon Nov 04 20:57:32 +0000 2019'
'Mon Nov 04 20:57:11 +0000 2019'
'Mon Nov 04 20:56:50 +0000 2019'
'Mon Nov 04 20:55:13 +0000 2019'
'Mon Nov 04 20:54:13 +0000 2019'
'Mon Nov 04 20:53:09 +0000 2019'
'Mon Nov 04 20:52:50 +0000 2019'
'Mon Nov 04 20:51:51 +0000 2019'
'Mon Nov 04 20:49:19 +0000 2019'
'Mon Nov 04 20:46:39 +0000 2019'
'Mon Nov 04 20:45:48 +0000 2019'
'Mon Nov 04 20:45:19 +0000 2019'
'Mon Nov 04 20:39:58 +0000 2019'
'Mon Nov 04 20:39:41 +0000 2019'
'Mon Nov 04 20:37:29 +0000 2019'
'Mon Nov 04 20:36:38 +0000 2019'
'Mon Nov 04 20:35:13 +0000 2019'
'Mon Nov 04 20:34:38 +0000 2019'
'Mon Nov 04 20:33:17 +0000 2019'
'Mon Nov 04 20:32:49 +0000 2019'
'Mon Nov 0

'Mon Nov 04 11:29:03 +0000 2019'
'Mon Nov 04 11:24:24 +0000 2019'
'Mon Nov 04 11:18:18 +0000 2019'
'Mon Nov 04 11:16:55 +0000 2019'
'Mon Nov 04 11:00:05 +0000 2019'
'Mon Nov 04 10:10:15 +0000 2019'
'Mon Nov 04 09:54:53 +0000 2019'
'Mon Nov 04 09:36:02 +0000 2019'
'Mon Nov 04 09:28:55 +0000 2019'
'Mon Nov 04 09:24:38 +0000 2019'
'Mon Nov 04 09:22:44 +0000 2019'
'Mon Nov 04 09:18:52 +0000 2019'
'Mon Nov 04 09:11:38 +0000 2019'
'Mon Nov 04 09:11:18 +0000 2019'
'Mon Nov 04 08:58:45 +0000 2019'
'Mon Nov 04 08:55:31 +0000 2019'
'Mon Nov 04 08:53:20 +0000 2019'
'Mon Nov 04 08:50:27 +0000 2019'
'Mon Nov 04 08:49:01 +0000 2019'
'Mon Nov 04 08:46:26 +0000 2019'
'Mon Nov 04 08:22:45 +0000 2019'
'Mon Nov 04 07:25:55 +0000 2019'
'Mon Nov 04 06:31:01 +0000 2019'
'Mon Nov 04 06:30:33 +0000 2019'
'Mon Nov 04 06:26:23 +0000 2019'
'Mon Nov 04 06:21:03 +0000 2019'
'Mon Nov 04 06:19:25 +0000 2019'
'Mon Nov 04 06:05:03 +0000 2019'
'Mon Nov 04 05:00:40 +0000 2019'
'Mon Nov 04 04:44:13 +0000 2019'
'Mon Nov 0

'Sun Nov 03 16:53:53 +0000 2019'
'Sun Nov 03 16:52:52 +0000 2019'
'Sun Nov 03 16:46:22 +0000 2019'
'Sun Nov 03 16:45:42 +0000 2019'
'Sun Nov 03 16:45:21 +0000 2019'
'Sun Nov 03 16:34:29 +0000 2019'
'Sun Nov 03 16:33:14 +0000 2019'
'Sun Nov 03 16:32:10 +0000 2019'
'Sun Nov 03 16:31:23 +0000 2019'
'Sun Nov 03 16:31:10 +0000 2019'
'Sun Nov 03 16:27:09 +0000 2019'
'Sun Nov 03 16:25:19 +0000 2019'
'Sun Nov 03 16:22:19 +0000 2019'
'Sun Nov 03 16:21:29 +0000 2019'
'Sun Nov 03 16:18:18 +0000 2019'
'Sun Nov 03 16:17:28 +0000 2019'
'Sun Nov 03 16:11:52 +0000 2019'
'Sun Nov 03 16:08:47 +0000 2019'
'Sun Nov 03 16:08:18 +0000 2019'
'Sun Nov 03 16:08:03 +0000 2019'
'Sun Nov 03 16:02:47 +0000 2019'
'Sun Nov 03 16:00:57 +0000 2019'
'Sun Nov 03 15:59:57 +0000 2019'
'Sun Nov 03 15:58:50 +0000 2019'
'Sun Nov 03 15:56:52 +0000 2019'
'Sun Nov 03 15:56:46 +0000 2019'
'Sun Nov 03 15:55:55 +0000 2019'
'Sun Nov 03 15:52:16 +0000 2019'
'Sun Nov 03 15:47:45 +0000 2019'
'Sun Nov 03 15:44:55 +0000 2019'
'Sun Nov 0

'Sat Nov 02 17:31:21 +0000 2019'
'Sat Nov 02 17:24:21 +0000 2019'
'Sat Nov 02 17:22:00 +0000 2019'
'Sat Nov 02 17:18:51 +0000 2019'
'Sat Nov 02 17:15:19 +0000 2019'
'Sat Nov 02 17:12:18 +0000 2019'
'Sat Nov 02 17:12:01 +0000 2019'
'Sat Nov 02 17:08:27 +0000 2019'
'Sat Nov 02 17:07:28 +0000 2019'
'Sat Nov 02 17:06:32 +0000 2019'
'Sat Nov 02 17:02:17 +0000 2019'
'Sat Nov 02 17:00:27 +0000 2019'
'Sat Nov 02 16:55:11 +0000 2019'
'Sat Nov 02 16:51:45 +0000 2019'
'Sat Nov 02 16:50:36 +0000 2019'
'Sat Nov 02 16:49:56 +0000 2019'
'Sat Nov 02 16:39:25 +0000 2019'
'Sat Nov 02 16:39:06 +0000 2019'
'Sat Nov 02 16:38:46 +0000 2019'
'Sat Nov 02 16:38:27 +0000 2019'
'Sat Nov 02 16:37:55 +0000 2019'
'Sat Nov 02 16:32:04 +0000 2019'
'Sat Nov 02 16:31:28 +0000 2019'
'Sat Nov 02 16:24:54 +0000 2019'
'Sat Nov 02 16:24:03 +0000 2019'
'Sat Nov 02 16:23:43 +0000 2019'
'Sat Nov 02 16:20:46 +0000 2019'
'Sat Nov 02 16:19:13 +0000 2019'
'Sat Nov 02 16:14:32 +0000 2019'
'Sat Nov 02 16:13:11 +0000 2019'
'Sat Nov 0

KeyboardInterrupt: 

In [37]:
print(tweet_collection.estimated_document_count())# number of tweets collected

user_cursor = tweet_collection.distinct("user.id")
print (len(user_cursor)) # number of unique Twitter users

1387
535


In [40]:
tweet_collection.create_index([("text", pymongo.TEXT)], name='text_index', default_language='english') # create a text index

tweet_cursor = tweet_collection.find({"$text": {"$search": "data mining"}}) # return tweets contain football
for document in tweet_cursor:
    try:
        print ('----')
#         pprint (document) # use pprint to print the entire tweet document
   
        print ('name:', document["user"]["name"]) # user name
        print ('text:', document["text"])         # tweets
    except:
        print ("***error in encoding")
        pass

----
name: TMJ-USA IT-DB Jobs
text: Interested in a job in #WestDesMoines, IA? This could be a great fit. Click the link in our bio to apply: Data Repo… https://t.co/P6Szjl4A8n
----
name: TMJ-IAD Jobs
text: Cognizant is looking for teammates like you. See our latest #BusinessMgmt job openings, including "Data Entry Clerk… https://t.co/5WHRWYM6j0
----
name: TMJ-IA IT Jobs
text: Join the Robert Half Technology team! See our latest #IT job openings, including "Data/Information Architect", via… https://t.co/z19XWUlv9k
----
name: TMJ-IAD IT Jobs
text: Want to land a job like "Data Management Consultant 3/eONE Operations Utility Team (DMI)" at Wells Fargo? Applicant… https://t.co/upbq3buX1A
----
name: Crédit Agricole TS
text: 💡📣 Salon #Datajob Paris: Rendez-vous le 22 novembreà  @datajobparis, la rencontre annuelle des métiers de la #data… https://t.co/cFoGgcil1k
----
name: Crédit Agricole RH
text: RT @CA_TS_: 💡📣 Salon #Datajob Paris: Rendez-vous le 22 novembreà  @datajobparis, la rencontre 